In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')
from google.colab import drive

drive.mount('/content/gdriv')
PATH ='/content/gdriv/My Drive/LG/data'

Mounted at /content/gdriv


In [ ]:
import datetime as dt

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    # minute  = int(x[10] + '0')
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

In [ ]:
train_err  = pd.read_csv(PATH+'/train_err_data.csv')
train_qual  = pd.read_csv(PATH+'/train_quality_data.csv')
train_prob  = pd.read_csv(PATH+'/train_problem_data.csv')

In [ ]:
train_err['time'] = train_err['time'].apply(make_datetime).dt.strftime('%Y%m%d%H%M%S')

In [ ]:
train_err['idx']=train_err['time'].astype(str)+train_err['user_id'].astype(str)

In [ ]:
idx=train_err['idx']

In [ ]:
new_idx=[]
a=0
for i in range(len(idx)):
  if i==0:
    new_idx.append(0)
    a=idx[0]
  else:
    if idx[i]==a:
      new_idx.append(new_idx[i-1])
    else: 
      a=idx[i]
      new_idx.append(new_idx[i-1]+1)    



In [ ]:
train_err['new_idx']=new_idx

In [ ]:
train_err

user_id            time  ...                  idx  new_idx
0           10000  20201101020000  ...  2020110102000010000        0
1           10000  20201101030000  ...  2020110103000010000        1
2           10000  20201101030000  ...  2020110103000010000        1
3           10000  20201101050000  ...  2020110105000010000        2
4           10000  20201101050000  ...  2020110105000010000        2
...           ...             ...  ...                  ...      ...
16554658    24999  20201130160000  ...  2020113016000024999  2550847
16554659    24999  20201130170000  ...  2020113017000024999  2550848
16554660    24999  20201130170000  ...  2020113017000024999  2550848
16554661    24999  20201130170000  ...  2020113017000024999  2550848
16554662    24999  20201130210000  ...  2020113021000024999  2550849

[16554663 rows x 8 columns]

In [ ]:
train_user_id_max = 2550850
train_user_id_min = 0
train_user_number = 2550850

In [ ]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['new_idx','errtype']].values
print(id_error)
error = np.zeros((train_user_number,42))  #15000,42의 어레이 형테를 0벡터로 만들고

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx,err - 1] += 1

error.shape
error

  0%|          | 0/16554663 [00:00<?, ?it/s]

[[      0      15]
 [      1      12]
 [      1      11]
 ...
 [2550848       4]
 [2550848       4]
 [2550849      15]]


100%|██████████| 16554663/16554663 [00:42<00:00, 385702.25it/s]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
train_err_group = train_err.groupby(['errtype', 'errcode']).count().reset_index()
train_err_group = train_err_group.reset_index()
train_err_group.rename(columns = {'index' : 'group_index'}, inplace=True)
train_err_group

group_index  errtype     errcode  ...   fwver     idx  new_idx
0               0        1           0  ...   21079   21079    21079
1               1        1     P-41001  ...       1       1        1
2               2        1     P-41007  ...      35      35       35
3               3        1    P-41007   ...     105     105      105
4               4        1     P-41011  ...       2       2        2
...           ...      ...         ...  ...     ...     ...      ...
2865         2865       40           0  ...  355729  355729   355729
2866         2866       40           1  ...  374649  374649   374649
2867         2867       41  NFANDROID2  ...  104138  104138   104138
2868         2868       42           2  ...     698     698      698
2869         2869       42           3  ...   33676   33676    33676

[2870 rows x 9 columns]

In [ ]:
train_err_group['errcodetype'] =  train_err_group["errtype"].map(str) + train_err_group["errcode"]
train_err_group

group_index  errtype     errcode  ...     idx  new_idx   errcodetype
0               0        1           0  ...   21079    21079            10
1               1        1     P-41001  ...       1        1      1P-41001
2               2        1     P-41007  ...      35       35      1P-41007
3               3        1    P-41007   ...     105      105     1P-41007 
4               4        1     P-41011  ...       2        2      1P-41011
...           ...      ...         ...  ...     ...      ...           ...
2865         2865       40           0  ...  355729   355729           400
2866         2866       40           1  ...  374649   374649           401
2867         2867       41  NFANDROID2  ...  104138   104138  41NFANDROID2
2868         2868       42           2  ...     698      698           422
2869         2869       42           3  ...   33676    33676           423

[2870 rows x 10 columns]

In [ ]:
train_err['errcodetype'] =  train_err["errtype"].map(str) + train_err["errcode"].map(str)

In [ ]:
train_err['errcodetype'].value_counts()

221                     1865361
23connection timeout    1835562
151                     1604951
161                     1249376
40                      1175592
                         ...   
3840101                       1
388738                        1
3828380                       1
3810684                       1
9C-11020                      1
Name: errcodetype, Length: 2871, dtype: int64

In [ ]:
train_err_merge = pd.merge(train_err, train_err_group[['errcodetype', 'group_index']], how='inner', on ='errcodetype')

In [ ]:
train_err_merge

user_id            time model_nm  ...  new_idx  errcodetype group_index
0           10000  20201101020000  model_3  ...        0          151         102
1           10000  20201101060000  model_3  ...        3          151         102
2           10000  20201102000000  model_3  ...        5          151         102
3           10000  20201102060000  model_3  ...        7          151         102
4           10000  20201103020000  model_3  ...        9          151         102
...           ...             ...      ...  ...      ...          ...         ...
16554657    24955  20201115120000  model_2  ...  2543940       385666        2032
16554658    24969  20201104110000  model_1  ...  2546322       386102        2376
16554659    24969  20201107120000  model_1  ...  2546357       384526        1583
16554660    24969  20201116010000  model_1  ...  2546443       383965        1376
16554661    24978  20201124000000  model_0  ...  2547824      3825999         532

[16554662 rows x 10 columns]

In [ ]:
id_error = train_err_merge[['new_idx','group_index']].values
print(id_error)
errorc = np.zeros((train_user_number,2870))  #15000,42의 어레이 형테를 0벡터로 만들고

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    errorc[person_idx ,err - 1] += 1

errorc.shape
errorc

[[      0     102]
 [      3     102]
 [      5     102]
 ...
 [2546357    1583]
 [2546443    1376]
 [2547824     532]]


In [ ]:
print(error.shape)
print(errorc.shape)
error=np.concatenate((error.T, errorc.T), axis=0)
error=error.T
print(error.shape)

In [ ]:
prob_user_id = sorted(train_prob.user_id.unique())

In [ ]:
train_err['prob'] = train_err['user_id'].isin(prob_user_id).astype('int64')

In [ ]:
train_prob = train_err.groupby(['new_idx', 'prob']).count().reset_index()

In [ ]:
pd.DataFrame(train_prob['prob']).value_counts()

In [ ]:
problem=train_prob['prob'].values

In [ ]:
train_x = error
train_y = problem
del error, problem
print(train_x.shape)
print(train_y.shape)

In [ ]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.1
# 파라미터 설정
params =      {
                'learning_rate' : 0.0001,
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train( 
                        params,
                        train_set       = d_train,
                        num_boost_round = 20,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 20
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

In [ ]:
print(np.mean(auc_scores))